# 하루 발전량 예측

In [12]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split # train/test데이터 나누기
from sklearn.preprocessing import LabelEncoder # 라벨링
from sklearn.metrics import accuracy_score

from sklearn.ensemble import RandomForestRegressor

In [2]:
# 데이터 불러오기
raw_df = pd.read_csv("data.csv")

In [ ]:
# RMSE = np.sqrt(np.sum((real - pred)**2)/real.size) 을 작게 만드는 것이 목표
# NaN : 데이터가 있어야 하는 데 missing된 것
# non : 없을 무

In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2405 entries, 0 to 2404
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Unnamed: 0     2405 non-null   int64  
 1   date           2405 non-null   object 
 2   hour           2405 non-null   int64  
 3   max_temp       2405 non-null   float64
 4   min_temp       2405 non-null   float64
 5   max_windspeed  2402 non-null   float64
 6   max_humid      2405 non-null   float64
 7   min_humid      2405 non-null   float64
 8   air            2405 non-null   int64  
 9   cloud          2405 non-null   int64  
 10  sunshine       2405 non-null   float64
 11  g_temp         2405 non-null   float64
 12  temp           2405 non-null   float64
 13  wind_velocity  2405 non-null   float64
 14  humidity       2405 non-null   int64  
 15  PV             2405 non-null   float64
dtypes: float64(10), int64(5), object(1)
memory usage: 300.8+ KB


In [14]:
# Unnamed 컬럼 제거
cl_df = raw_df.iloc[:,1:]
cl_df

,date,hour,max_temp,min_temp,max_windspeed,max_humid,min_humid,air,cloud,sunshine,g_temp,temp,wind_velocity,humidity,PV
0,2020-09-28,7,15.8,15.8,NaN,88.0,88.0,1537,3,0.0,16.0,15.8,1.6,88,50.000000
1,2020-09-28,8,17.2,17.2,NaN,82.0,82.0,1434,0,0.0,16.5,17.2,3.1,82,97.891492
2,2020-09-28,9,20.3,20.3,NaN,70.0,70.0,1561,4,0.7,18.1,20.3,2.8,70,117.995666
3,2020-09-28,10,24.5,22.4,32.72,55.9,47.5,1701,6,1.0,21.5,22.0,2.4,58,170.700000
4,2020-09-28,11,25.0,23.2,35.26,51.3,42.2,1685,6,0.9,30.5,22.9,3.3,54,179.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,2021-01-13,10,6.1,2.9,2.16,60.2,47.4,1316,0,1.0,-0.6,0.8,1.0,72,115.800000
2401,2021-01-13,11,9.7,6.3,3.48,47.6,37.4,1339,0,1.0,0.2,4.7,1.5,58,143.200000
2402,2021-01-13,12,11.1,9.7,4.39,38.7,34.8,1160,0,1.0,5.2,9.1,3.5,46,150.400000
2403,2021-01-13,13,11.7,10.5,5.57,37.0,29.1,1109,0,1.0,7.2,9.8,4.4,46,142.100000


In [185]:
# PV <= 0 인 row 삭제
cl_data = cl_df.iloc[~np.array(cl_df.PV<=0), :]

cl_data

,date,hour,max_temp,min_temp,max_windspeed,max_humid,min_humid,air,cloud,sunshine,g_temp,temp,wind_velocity,humidity,PV
0,2020-09-28,7,15.8,15.8,NaN,88.0,88.0,1537,3,0.0,16.0,15.8,1.6,88,50.000000
1,2020-09-28,8,17.2,17.2,NaN,82.0,82.0,1434,0,0.0,16.5,17.2,3.1,82,97.891492
2,2020-09-28,9,20.3,20.3,NaN,70.0,70.0,1561,4,0.7,18.1,20.3,2.8,70,117.995666
3,2020-09-28,10,24.5,22.4,32.72,55.9,47.5,1701,6,1.0,21.5,22.0,2.4,58,170.700000
4,2020-09-28,11,25.0,23.2,35.26,51.3,42.2,1685,6,0.9,30.5,22.9,3.3,54,179.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2400,2021-01-13,10,6.1,2.9,2.16,60.2,47.4,1316,0,1.0,-0.6,0.8,1.0,72,115.800000
2401,2021-01-13,11,9.7,6.3,3.48,47.6,37.4,1339,0,1.0,0.2,4.7,1.5,58,143.200000
2402,2021-01-13,12,11.1,9.7,4.39,38.7,34.8,1160,0,1.0,5.2,9.1,3.5,46,150.400000
2403,2021-01-13,13,11.7,10.5,5.57,37.0,29.1,1109,0,1.0,7.2,9.8,4.4,46,142.100000


In [186]:
cl_data.reset_index(inplace=True)

In [187]:
cl_data

,index,date,hour,max_temp,min_temp,max_windspeed,max_humid,min_humid,air,cloud,sunshine,g_temp,temp,wind_velocity,humidity,PV
0,0,2020-09-28,7,15.8,15.8,NaN,88.0,88.0,1537,3,0.0,16.0,15.8,1.6,88,50.000000
1,1,2020-09-28,8,17.2,17.2,NaN,82.0,82.0,1434,0,0.0,16.5,17.2,3.1,82,97.891492
2,2,2020-09-28,9,20.3,20.3,NaN,70.0,70.0,1561,4,0.7,18.1,20.3,2.8,70,117.995666
3,3,2020-09-28,10,24.5,22.4,32.72,55.9,47.5,1701,6,1.0,21.5,22.0,2.4,58,170.700000
4,4,2020-09-28,11,25.0,23.2,35.26,51.3,42.2,1685,6,0.9,30.5,22.9,3.3,54,179.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,2400,2021-01-13,10,6.1,2.9,2.16,60.2,47.4,1316,0,1.0,-0.6,0.8,1.0,72,115.800000
1198,2401,2021-01-13,11,9.7,6.3,3.48,47.6,37.4,1339,0,1.0,0.2,4.7,1.5,58,143.200000
1199,2402,2021-01-13,12,11.1,9.7,4.39,38.7,34.8,1160,0,1.0,5.2,9.1,3.5,46,150.400000
1200,2403,2021-01-13,13,11.7,10.5,5.57,37.0,29.1,1109,0,1.0,7.2,9.8,4.4,46,142.100000


In [188]:
cl_data.fillna(cl_data.max_windspeed.mean(),inplace=True)

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/frame.py:4462: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


In [190]:
cl_data

,index,date,hour,max_temp,min_temp,max_windspeed,max_humid,min_humid,air,cloud,sunshine,g_temp,temp,wind_velocity,humidity,PV
0,0,2020-09-28,7,15.8,15.8,30.063528,88.0,88.0,1537,3,0.0,16.0,15.8,1.6,88,50.000000
1,1,2020-09-28,8,17.2,17.2,30.063528,82.0,82.0,1434,0,0.0,16.5,17.2,3.1,82,97.891492
2,2,2020-09-28,9,20.3,20.3,30.063528,70.0,70.0,1561,4,0.7,18.1,20.3,2.8,70,117.995666
3,3,2020-09-28,10,24.5,22.4,32.720000,55.9,47.5,1701,6,1.0,21.5,22.0,2.4,58,170.700000
4,4,2020-09-28,11,25.0,23.2,35.260000,51.3,42.2,1685,6,0.9,30.5,22.9,3.3,54,179.900000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1197,2400,2021-01-13,10,6.1,2.9,2.160000,60.2,47.4,1316,0,1.0,-0.6,0.8,1.0,72,115.800000
1198,2401,2021-01-13,11,9.7,6.3,3.480000,47.6,37.4,1339,0,1.0,0.2,4.7,1.5,58,143.200000
1199,2402,2021-01-13,12,11.1,9.7,4.390000,38.7,34.8,1160,0,1.0,5.2,9.1,3.5,46,150.400000
1200,2403,2021-01-13,13,11.7,10.5,5.570000,37.0,29.1,1109,0,1.0,7.2,9.8,4.4,46,142.100000


In [240]:
from sklearn.preprocessing import StandardScaler

In [244]:
scaler = StandardScaler()
scaler.fit(cl_data[['sunshine']])

sunshine_n = scaler.transform(cl_data[['sunshine']])
sunshine_n

array([[-1.25109821],
       [-1.25109821],
       [ 0.32452525],
       ...,
       [ 0.99979245],
       [ 0.99979245],
       [ 0.99979245]])

In [245]:
sunshine_scaled = pd.DataFrame(data=sunshine_n, columns=cl_data.sunshine)

ValueError: Shape of passed values is (1202, 1), indices imply (1202, 1202)

In [229]:
feature_cols = ['hour','max_temp','max_windspeed','max_humid','air','cloud','sunshine','g_temp','temp','wind_velocity','humidity']
y_col = ['PV']

X = cl_data[feature_cols]
y = cl_data[y_col]

In [235]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [236]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

<ipython-input-236-d3f886df783a>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf_model.fit(X_train, y_train)


RandomForestRegressor()

In [237]:
rf_pred = rf_model.predict(X_test)

In [238]:
RMSE = np.sqrt(np.sum((np.array(y_test) - np.array(rf_pred))**2)/y_test.size)

In [239]:
RMSE

1581.352030212107